In [1]:
import datasets
import torch
from torch import nn
from transformers import MBartModel, MBartTokenizer, MBartConfig
from datasets import concatenate_datasets, load_from_disk
from torch.optim import *
from torch.utils import data
from torch.utils.data import DataLoader, TensorDataset
from MBartPreTrainingDataset import MBartPreTrainingDataset
from datasets import load_dataset
from MBart import MBart
from accelerate import Accelerator
from MBartDataset import MBartDataset

In [2]:
def model_size(model):
    param_size = 0
    for param in model.parameters():
        param_size += param.nelement() * param.element_size()
    buffer_size = 0
    for buffer in model.buffers():
        buffer_size += buffer.nelement() * buffer.element_size()

    size_all_mb = (param_size + buffer_size) / 1024 ** 2
    return size_all_mb

In [3]:
#pre_train_ds = load_dataset("text", data_files={"train": ["/data/n.dallanoce/cc100/en.txt"]},
#                            cache_dir="/data/n.dallanoce/cc100/hugg_en", split='train', ignore_verifications=True)

tok_en = MBartTokenizer.from_pretrained("facebook/mbart-large-cc25", src_lang="en_XX")

#pre_train_ds = MBartPreTrainingDataset(pre_train_ds, tok_en, "en")

mbart_config = MBartConfig(encoder_layers=6, decoder_layers=6,
                           encoder_ffn_dim=2048, decoder_ffn_dim=2048,
                           encoder_attention_heads=8, decoder_attention_heads=8,
                           d_model=512, max_length=128, vocab_size= tok_en.vocab_size)

#accelerator = Accelerator(mixed_precision='fp16', gradient_accumulation_steps=1)
model: MBart = MBart(mbart_config, device_ids=[3])
print(model_size(model))

#dataset_loaded = load_from_disk("europarl_eng_tokenized")
#pre_train_load = DataLoader(pre_train_ds, batch_size=8, drop_last=True, shuffle=False, pin_memory=True, num_workers=4)

661.6861991882324


In [20]:
param_tot = 0
for param in model.parameters():
    param_tot += param.nelement()
print(param_tot)

610879488


In [4]:
model.load_state_dict(torch.load("mbart_v2_test.pt"))

<All keys matched successfully>

In [10]:
from transformers import MBartForConditionalGeneration

tok_en = MBartTokenizer.from_pretrained("facebook/mbart-large-cc25", src_lang="en_XX")
model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50")

In [7]:
cuda_dev = "cuda:3"
model = model.to(cuda_dev)

In [8]:
sentence = "I <mask> the book, and it's certainly deserving. </s> en_XX"
test_ids = tok_en([sentence], add_special_tokens=False, return_tensors="pt")["input_ids"]
logits = model(test_ids.to(cuda_dev)).logits
masked_index = (test_ids[0] == tok_en.mask_token_id).nonzero().item()
probs = logits[0, masked_index].softmax(dim=0)
values, predictions = probs.topk(5)
tok_en.decode(predictions).split()

["have'", 'had', 'am', 'think']

In [9]:
test_ids

tensor([[     2,  72607,      7, 250026,  10932,     70,   8999,      5,      2,
         250004]])

In [12]:
sentence = "I also thought that Black River was very impressive and tightly written, so that's my dark horse candidate for this year's prize."
test_ids = tok_en([sentence], add_special_tokens=True, return_tensors="pt")["input_ids"]
logits = model(test_ids.to(cuda_dev)).logits
masked_index = (test_ids[0] == tok_en.mask_token_id).nonzero().item()
probs = logits[0, masked_index].softmax(dim=0)
values, predictions = probs.topk(5)
tok_en.decode(predictions).split()

['written', 'thought', 'curious', 'I', 'finalist']

In [7]:
test_ids

tensor([[ 72607,      7, 250026,  10932,     70,   8999,      5,      2, 250004]])